_Anmerkung: Ggf. müssen sie nach dem Öffnen des Notebooks noch einmal auf 'Kernel > Restart & Run All' clicken_

<h1>Demonstrator: Abfragen mit SPARQL</h1>

In diesem Demonstrator werden wir zeigen, wie SPARQL Abfragen genutzt werden können, 
um aus Open Data Beständen weiteres Wissen zu ziehen.

![SPARQL Logo](./media/SPARQL.png)

<h2> Übersicht </h2>

Im Verlauf dieses Vortrages werden wir nochmal die Grundlagen von SPARQL-Abfragen darstellen, 
sowie danach erläutern, wie sich derart gewonnene Daten zur Erweiterung eines bestehenden Datenbestandes Nutzen lassen. <br><br>
Idealerweise können sie am Ende dieses Demonstrators beim ihren nächsten Projekt ebenfalls Daten über Open Data im RDF-Format erweitern. <br>
In diesem Vortrag kommen daher folgende Themen vor:

    - Was ist SPARQL 
    - SPARQL Queries
    - Erweiterung einer DB mit Open Data
        - Suche in der DB
        - Geeignete Attribute zur Identifikation über mehrere DB
        - Erweiterung der DB
    - Welche Daten sind (in der erweiterten DB) nicht vorhanden
    - Fazit

<h2> Was ist SPARQL? </h2>

Bei SPARQL handelt es sich um die **S**PARQL **P**rotocol **A**nd **R**DF **Q**uery **L**anguage. <br>
Es ist also sowohl ein Protokoll als auch eine Abfragesprache für das Resource Description Framework. <br>
Während SQL die Abfragesprache für relationale Datenbanken ist, ist SPARQL die geläufige Abfragesprache für RDF-Daten, welche nicht als Tabellen sondern als Graph mit gelabelten Kanten zu verstehen sind.
RDF steht für **R**esource **D**escription **F**ramework, ein vom W3C standardisiertes, XML-basiertes Format zur Beschreibung von Datenobjekten und ihren Beziehungen zueinander.



<h2> SPARQL Queries </h2>

Eine SPARQL Abfrage ähnelt in ihren Augen möglicherweise einer SQL-Abfrage, jedoch sind die Gemeinsamkeiten nur oberflächlich.
Abfragen besitzen einen SELECT-Part, der die resultierenden Variablen (von der Struktur den Spalten in SQL entsprechend) wiedergibt und einem WHERE-Part welcher definiert, welche Eigenschaften die resultierenden Filme alle gemeinsam besitzen.

In [1]:
from data.sparql import query, table

table(query("""
SELECT ?movie
WHERE
{
 ?movie wdt:P31 wd:Q11424 #wdt:P31 = instance of; wd:Q11424 = film
}
LIMIT 10
"""))

movie
http://www.wikidata.org/entity/Q372
http://www.wikidata.org/entity/Q593
http://www.wikidata.org/entity/Q595
http://www.wikidata.org/entity/Q1365
http://www.wikidata.org/entity/Q2201
http://www.wikidata.org/entity/Q2345
http://www.wikidata.org/entity/Q2875
http://www.wikidata.org/entity/Q3092
http://www.wikidata.org/entity/Q3187
http://www.wikidata.org/entity/Q3208


Diese Query liefert die ersten 10 Filme, welche in Wikidata gelistet sind.
Zwar können sie auf die Einträge klicken, um zu sehen, um was es sich handelt, jedoch ist dieses Format noch nicht sonderlich leserlich.

Um statt der Objekte selbst ihre Label zu erhalten, erweitern wir die vorherige Query. Diesmal soll nicht der Film selbst, sondern sein Titel angezeigt werden.

In [2]:
from data.sparql import query, table

table(query(
"""
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424 .
 ?movie wdt:P1448 ?title       
} 
LIMIT 10
"""
))    

| title |
| --- |
| ...men Olsenbanden var ikke død |
| Le guerriere dal seno nudo |
| La vendetta dei gladiatori |
| Zinebi Festival Internacional de Cine Documental y Cortometraje de Bilbao |
| Relatives |
| Börn |
| Syurpriz |
| Petualang Tak Kenal Menyerah |
| طلق صناعي |
| Brexit (2019) |


'showing 10 results'

Wie sie sehen, muss nicht jede Variable auch ausgegeben werden.
Hier sieht man nur den Titel, aber nicht das Film-Objekt.

Zudem ist dies die erste Query, bei der mehrere Kriterien (nämlich Objekt ?movie ist vom Typ Film und Object ?movie hat Titel ?title.
Jedes dieser logischen Tripletts wird durch einen Punkt vom nächsten getrennt.
Um etwas Schreibarbeit zu sparen, kann auch 

```
SELECT ?title
WHERE
{
 ?movie wdt:P31 wd:Q11424;
        wdt:P1448 ?title       
} 
LIMIT 10
```

geschreiben werden. Hier wird durch das Semikolon ";" das Subjekt aus dem letzten Triplet für das Nächste wiederverwendet.

Schreiben sie ebenfalls eine Abfrage, welche die IMDB ID dreier Filme ausgibt.
Das Prädikat für 'hat IMDB ID' lautet 'wdt:P345', bei der IMDB ID handelt es sich um eine ID der 'Internet Movie Database'.

In [3]:
# Tragen sie hier ihren SPARQL-Code ein und klicken sie auf RUN, um ihn auszutesten.
# Denken sie daran, ggf. über 'from data.sparql import query, table' die notwendigen Funktionen zu importieren,
# sollte es sie hier zu einem Fehler kommen.








<h2> Erweiterung einer DB mit Open Data </h2>

Sie haben nun einen Überblick über die grundlegende Struktur von SPARQL Abfragen bekommen.<br>
Die durch SPARQL verfügbar gemachten Daten können verwendet werden, 
um beispielsweise zusätzliche Informationen zu denen einer Datenbank hinzuzufügen. <br><br>

In diesem Notebook handelt es sich hierbei um eine MongoDB, welche eine vielzahl an Filmen enthält.
Die Daten entstammen den bekannten *movie_metadata.csv* File.
Zur Rekapitulation: Dieser Datensatz enthält folgende Informationen zu Filmen bis zum Jahr 2020:

- index (Integer, n-ter Eintrag innerhalb des Datensatzes)
- title
- adult (Boolean, ob der Film FSK 18 ist)
- belongs_to_collection
- budget
- genres
- homepage
- id (Integer, ID innerhalb des Datensatz)
- imdb_id (String, ID innerhalb der Internet Movie Database)
- original_language
- original_title
- overview (String, sehr kurze Textzusammenfassung der Handlung)
- popularity
- poster_path
- production_companies
- production_countries
- release_date
- revenue
- runtime
- spoken_languages
- status (String, Released, produziert aber nicht veröffentlicht, etc.)
- tagline (String, Slogan, Untertitel des Films)
- video
- vote_average
- vote_count


Die Datenbank enthält also bereits viele Informationen.
Titel, Genre, Veröffentlichungszeitpunkt und ähnliches ist enthalten. <br>
Jedoch fehlen auch einige Informationen in den bisherigen Daten.
Beispielsweise: <br>
- Welche Schauspieler:innen haben in den Film gespielt? (Bzw. wer hat allgemein am Film mitgewirkt) 
- In welchen Ländern wurde der Film veröffentlicht?
- Welche Altersfreigabe gibt es **pro Land**?

Diese im Original-Datensatz nicht vorhandenen Informationen können gegebenenfalls über Open Data im Netz in Erfahrung gebracht werden. 
Dies geschieht im folgenden.
In der Datenbank befindet sich beispielsweise zum Film Iron Man folgendes:

In [4]:
import data.movies as movies

iron_man_movies = movies.get_by_title('Iron Man')

table(iron_man_movies)

Key,Value
index,12588
adult,False
belongs_to_collection,Iron Man Collection
budget,140000000
genres,"Action,Science Fiction,Adventure"
homepage,http://www.ironmanmovie.com/
id,1726
imdb_id,tt0371746
original_language,en
original_title,Iron Man


Um diesen Film, sowie die anderen, in der Datenbank befindlichen Einträge um zusätzliche Informationen zu erweitern, muss zunächst ein geeigneter eindeutiger Identifier für einen Film gefunden werden. <br>
Dazu eignen sich einige Einträge eines Films besser als andere. <br><br>

Frage: Welche der Eigenschaften des (oben sichtbaren) Filmes haltet ihr für sinvoll um online nach Infos zu suchen?<br>
Und warum?





<h3> Suche nach Namen: </h3>

Im folgenden suchen wir zuerst Filme nach Namen

In [ ]:
from data.sparql import query, data, table, entity_to_json

#movie is instance (P31) of film (Q11424)
#movie has label ?titel

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
"""
))


Hier haben wir viele Ergebnisse, welche ggf. hilfreich sind, wenn der Nutzer im allgemeinen nach Schlagworten sucht.
Für die Exploration eines Datensatzes sinnvoll, für die Erweiterung von Daten jedoch nicht geeignet. Hier wollen wir eher ein eindeutiges Attribut haben und der Name allein (ohne weitere Merkmale wie Veröffentlichungsjahr und Regisseur) ist dies nicht.

<h3> Suche nach imdb_id: </h3>

Die IMDB enthält viele Informationen zu Filmen und listet Filme unter der eigenen ID. <br>
Diese IMDB ID kann genutzt werden, um ggf. mehr Daten aus Open Data herauszufinden.

In [ ]:
from data.sparql import query, table

table(query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        wdt:P345 ?imdb_id .
 FILTER CONTAINS (?imdb_id, "tt0371746")
} 
LIMIT 1
"""
))



Auf WikiData finden wir über die IMDB ID ein Datenobjekt. <br>
Als Nutzer können sie auf den Link dieses Datenobjektes klicken und sich die Wikidata-Seite ansehen.
Zur maschinellen Verarbeitung kann aber auch das Objekt in als JSON angefragt und dann weiter verarbeitet werden.

In [ ]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

# a bit hacky, but it retrieves the id from the wikidata URI, which has the form https://www.wikidata.org/wiki/{id}
url = data(result)[0]['movie']
entity_id = url.split("/").pop()

print(url)
entity_to_json(entity_id)

Hier erkennt man, dass wir in diesem JSON Objekt eine große Menge an Daten zum Film erhalten. 
Wir können hier den Film-Titel in vielen Sprachen angeben, herausfinden, welcher Direktor für den Schnitt des Filmes verantwortlich ist, welche unterschiedlichen Jugendschutz-Label ein Film hat, etc.

<h3> Einfügen aller Mitwirkenden eines Films </h3>

In [4]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT *
WHERE
{
 ?movie wdt:P31 wd:Q11424 ;
        rdfs:label ?title ;
        wdt:P345 ?imdb_id .
  FILTER CONTAINS(?title, "Iron Man")
  FILTER LANGMATCHES(lang(?title),'en')
} 
LIMIT 1
"""
)

Einige der Eigenschaften sind jedoch über ihre Wikidata-ID aufgelistet. Diese muss aufgelöst werden, um menschenverständliche Daten zu erhalten, also z.B. das es sich bei 'P345' um eine 'IMDB ID' handelt. 

In [16]:
from data.sparql import query, data, table, entity_to_json

result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746". #P345 = IMDb-ID 
    ?title ?property ?cast .     # übernehme alle Parameter des Films als Vairablen
    ?cast wdt:P31 wd:Q5 .        # P31 = isInstanceOf Q5: human

    # get the property label
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)

table (result)

| titleLabel |castLabel |property |propLabel |
| --- | --- | --- | --- |
| Iron Man |Matthew Libatique |http://www.wikidata.org/prop/direct/P344 |director of photography |
| Iron Man |Gwyneth Paltrow |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ramin Djawadi |http://www.wikidata.org/prop/direct/P86 |composer |
| Iron Man |Clark Gregg |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Shaun Toub |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Robert Downey Jr. |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Samuel L. Jackson |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jeff Bridges |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Stan Lee |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Leslie Bibb |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Paul Bettany |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jon Favreau |http://www.wikidata.org/prop/direct/P57 |director |
| Iron Man |Jon Favreau |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Terrence Howard |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tom Morello |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Faran Tahir |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Joshua Harto |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Mark Fergus |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Tim Guinee |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Bill Smitrovich |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Nazanin Boniadi |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Meera Simhan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Sayed Badreya |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Hawk Ostby |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Art Marcum |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Matt Holloway |http://www.wikidata.org/prop/direct/P58 |screenwriter |
| Iron Man |Kevin Feige |http://www.wikidata.org/prop/direct/P162 |producer |
| Iron Man |Avi Arad |http://www.wikidata.org/prop/direct/P162 |producer |
| Iron Man |Will Lyman |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |America Olivo |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Peter Billingsley |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Adam Harrington |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ahmed Ahmed |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jim Cramer |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Lauren Scyphers |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Lauren Scyphers |http://www.wikidata.org/prop/direct/P175 |performer |
| Iron Man |Marco Khan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Mike Cochrane |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ricki Noel Lander |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Gabrielle Tuite |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tim Griffin |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |James Bethea |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Sarah Cahill |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Fahim Fazli |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Russell Richardson |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Micah Hauptman |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Masha Lund |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Patrick O'Connell |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Daston Kalili |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ido Mor |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Kevin Foster |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Garret Noël |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Eileen Weisinger |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Lana Kinnear |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Nicole Lindeblad |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Zorianna Kit |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ben Newmark |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Jeannine Kaspar |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Stacy Stas Hurst |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Stacy Stas Hurst |http://www.wikidata.org/prop/direct/P175 |performer |
| Iron Man |Gerard Sanders |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Tim Rigby |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Thomas Craig Plumer |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Robert Berkman |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Frank Nyi |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Marvin Jordan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Donna Evans |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Reid Harper |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Ava Rose Williams |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Summer Kylie Remington |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Vladimir Kubr |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Callie Croughwell |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Javan Tahir |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Sahar Bibiyan |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Justin Rex |http://www.wikidata.org/prop/direct/P161 |cast member |
| Iron Man |Stan Lee |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Jon Favreau |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |J. Michael Riva |http://www.wikidata.org/prop/direct/P2554 |production designer |
| Iron Man |Dan Lebental |http://www.wikidata.org/prop/direct/P1040 |film editor |
| Iron Man |Peter Billingsley |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |David Maisel |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Louis D'Esposito |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Ari Arad |http://www.wikidata.org/prop/direct/P1431 |executive producer |
| Iron Man |Laura Jean Shannon |http://www.wikidata.org/prop/direct/P2515 |costume designer |
| Iron Man |Rebecca Bentjen |http://www.wikidata.org/prop/direct/P2515 |costume designer |


'showing 85 results'

Die so abgerufene Liste an Schauspielern bzw. Mitwirkenden dieses Films wird nun dem Datensatz der lokalen Datenbank hinzugefügt. Dazu werden die Daten zunächst aufbereitet und dann in dem Datensatz gespeichert.

Der Datensatz enthält daraufhin die Schauspieler als ein weiteres Attribut:

In [17]:
from data.sparql import query, data, table, entity_to_json
import data.movies as movies


result = query(
"""
SELECT ?titleLabel ?castLabel ?property ?propLabel
WHERE {
    ?title wdt:P345 "tt0371746". #P345 = IMDb-ID 
    ?title ?property ?cast .     # übernehme alle Parameter des Films als Vairablen
    ?cast wdt:P31 wd:Q5 .        # P31 = isInstanceOf Q5: human

    # get the property label
    ?prop wikibase:directClaim ?property .

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
)

dict = {"cast_members": [{"name": cast["castLabel"]["value"], "role": cast["propLabel"]["value"]} for cast in result["results"]["bindings"]]}
#print (dict)


movies.add_data_by_title("Iron Man", dict)

table(movies.get_by_title('Iron Man'))

KeyError: 'head'

Dieser Vorgang kann nun automatisiert für alle Einträge der lokalen Datenbank durchlaufen werden. 

In [ ]:
from data.sparql import query, data, table, entity_to_json
import json
import data.movies as movies

def wikidata_query (imdb_id):
    return query(
    f"""
    SELECT ?titleLabel ?castLabel ?property ?propLabel
    WHERE {{
        ?title wdt:P345 "{imdb_id}".
        # take all claims on this movie
        ?title ?property ?cast .
        # that involve a human
        ?cast wdt:P31 wd:Q5 .

        # get the property label
        # see https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries#Adding_labels_for_properties
        hint:Query hint:optimizer "None" .
        ?prop wikibase:directClaim ?property .

        SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    """
    )

def extract_cast (wikidata_result):
    dict = {"cast_members": []}
    for cast_member in result["results"]["bindings"]:
        cast = {
            "Name": cast_member["castLabel"]["value"],
            "Role": cast_member["propLabel"]["value"]
        }
        dict["cast_members"].append(cast)
    return dict


#load all movies from local db (limited to 5):
local_movies = movies.get_all(10000)
for movie in local_movies:
    imdb_id = movie["imdb_id"]
    title = movie["original_title"]
    movie_id = movie["id"]
 
    #Query an 
    result = wikidata_query(imdb_id)
    
    cast_dict = extract_cast(result)
    
    movies.add_data_by_title(title, cast_dict)
    #print(json.dumps(movies.get_by_title(title), indent=1))
    
    print(f"handled movie: '{title}' with imdb {imdb_id}")


In [ ]:
import json
import data.movies as movies

print(json.dumps(movies.get_by_title("Dingo"), indent=1))




<h2> Welche Daten sind nicht vorhanden </h2>

Frage: Wie kann man hier die Performance erhöhen

Zu welchen Problemen kann es kommen und wie kann auf diese reagiert werden?

Frage:
Welche Daten kann man nicht einfach über Wikidata beziehen?
Bsp. 
- Wie lange liefen Filme in einem Kino?
- In Welchen Kinos lief ein Film?

<h2> Fazit </h2>